# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/anna_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-3.66,97,100,5.14,CA,1730159916
1,1,huangshan,29.7114,118.3125,14.71,86,100,1.25,CN,1730159918
2,2,wailua homesteads,22.0669,-159.3780,27.44,82,40,9.26,US,1730159919
3,3,waitangi,-43.9535,-176.5597,12.12,87,49,6.84,NZ,1730159920
4,4,bredasdorp,-34.5322,20.0403,13.94,96,25,2.74,ZA,1730159922


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df["Max Temp"] > 18) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,newman,37.3138,-121.0208,18.36,49,39,3.58,US,1730159928
29,29,crane,31.3974,-102.3501,26.75,19,13,9.29,US,1730159951
50,50,al artawiyah,26.5053,45.3444,20.69,34,71,2.97,SA,1730159980
73,73,jerez de garcia salinas,22.6500,-103.0000,19.90,26,0,3.61,MX,1730160006
76,76,al kharj,24.1550,47.3346,21.32,31,67,2.34,SA,1730160009


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,yellowknife,CA,62.4560,-114.3525,97,
1,huangshan,CN,29.7114,118.3125,86,
2,wailua homesteads,US,22.0669,-159.3780,82,
3,waitangi,NZ,-43.9535,-176.5597,87,
4,bredasdorp,ZA,-34.5322,20.0403,96,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yellowknife - nearest hotel: No hotel found
huangshan - nearest hotel: 丰华旅馆
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
waitangi - nearest hotel: Hotel Chathams
bredasdorp - nearest hotel: Victoria Hotel
port-aux-francais - nearest hotel: Keravel
blackmans bay - nearest hotel: Villa Howden
barma - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
newman - nearest hotel: No hotel found
iron river - nearest hotel: AmericInn
edinburgh of the seven seas - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
port saint john's - nearest hotel: Outback Inn
east london - nearest hotel: No hotel found
madimba - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
afaahiti - nearest hotel: Omati Lodge
papatowai - nearest hotel: No hotel found
nova granada - nearest hotel: Hotel Granada
grytviken - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
ilulissa

,City,Country,Lat,Lng,Humidity,Hotel Name
0,yellowknife,CA,62.4560,-114.3525,97,No hotel found
1,huangshan,CN,29.7114,118.3125,86,丰华旅馆
2,wailua homesteads,US,22.0669,-159.3780,82,Hilton Garden Inn Kauai Wailua Bay
3,waitangi,NZ,-43.9535,-176.5597,87,Hotel Chathams
4,bredasdorp,ZA,-34.5322,20.0403,96,Victoria Hotel
...,...,...,...,...,...,...
545,coahuayana de hidalgo,MX,18.7000,-103.6583,80,No hotel found
546,aitape,PG,-3.1374,142.3475,86,No hotel found
547,kisangani,CD,0.5167,25.2000,87,Hotel Kisangani
548,nikolayevsk-on-amure,RU,53.1406,140.7300,81,Север


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)